In [19]:
from tinybig.config import rpn_config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'diabetes_configs'
config_obj = rpn_config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'mps', 'random_seed': 1234, 'data_configs': {'data_class': 'tinybig.data.diabetes', 'data_parameters': {'name': 'diabetes', 'train_batch_size': 1000, 'test_batch_size': 1000}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 3000, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.0005, 'weight_decay': 1e-05}}, 'loss_configs': {'loss_class': 'torch.nn.MSELoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'device': 'mps', 'model_parameters': {'name': 'reconciled_polynomial_network', 'depth': 1, 'depth_alloc': [1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_parameters': {'name': 'rpn_layer', 'm': 8, 'n': 1, 'width': 1, 'fusion_strategy': 'average', 'width_alloc': [1], 'head_configs': [{'head_class': 'tinybig.module.rpn_h

In [20]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  mps ; random_seed:  1234


In [21]:
print('instantiating objects from config...')
object_dict = config_obj.instantiate_object_from_config(config['configurations'])
data_obj, model_obj, learner_obj, metric_obj, result_obj = [object_dict[name] for name in ['data', 'model', 'learner', 'metric', 'result']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
combinatorial_normal_expansion initialization
parameter num:  44


In [22]:
print('loading dataset...')
data_loader = data_obj.load(train_percentage=0.9, normalize_X=True)

loading dataset...


In [23]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device, metric=metric_obj, disable_tqdm=True, display_step=100)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...
Epoch: 0, Test Loss: 0.5340700745582581, Test Score: 0.5340701268297842, Time Cost: 0.006468772888183594
Epoch: 100, Test Loss: 0.29197242856025696, Test Score: 0.2919724107636688, Time Cost: 0.0055277347564697266
Epoch: 200, Test Loss: 0.2766035795211792, Test Score: 0.27660356945729825, Time Cost: 0.005342245101928711
Epoch: 300, Test Loss: 0.26146456599235535, Test Score: 0.2614645354786587, Time Cost: 0.005430936813354492
Epoch: 400, Test Loss: 0.24704204499721527, Test Score: 0.24704204207501665, Time Cost: 0.005308866500854492
Epoch: 500, Test Loss: 0.23358845710754395, Test Score: 0.233588461016266, Time Cost: 0.005259990692138672
Epoch: 600, Test Loss: 0.22124238312244415, Test Score: 0.22124238750083064, Time Cost: 0.005246162414550781
Epoch: 700, Test Loss: 0.21007299423217773, Test Score: 0.21007298251503007, Time Cost: 0.0055370330810546875
Epoch: 800, Test Loss: 0.2001015841960907, Test Score: 0.2001015802964822, Time Cost: 0.005261659622192383
Epoch: 900

In [24]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [25]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
mse 0.15247175085902862


In [26]:
from tinybig.metric import accuracy

y_rounded_label = [[round(y[0])] for y in test_result['y_score']]
test_result['y_pred'] = y_rounded_label

acc_metric = accuracy('accuracy_metric')
print('evaluating rounded prediction labels...')
print(acc_metric.__class__.__name__, acc_metric.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating rounded prediction labels...
accuracy 0.8051948051948052


In [27]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
